<a href="https://colab.research.google.com/github/ipeirotis/introduction-to-databases/blob/master/session5/practice_queries_join_and_aggregation_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 5: Practice Aggregate+Join Queries: Solutions


In this segment we will connect to the *Restaurants* database.

## Setup

In [0]:
!apt-get install python3-mysqldb
!pip install sqlalchemy sql_magic

In [0]:
# This code creates a connection to the database
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'restaurants',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')

engine = create_engine(conn_string)
con = engine.connect()

In [4]:
%reload_ext sql_magic

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
%config SQL.conn_name = 'engine'

## Question 1: Output the restaurant name together with the number of reviews submitted for this restaurant.

Let's run first the join query, and look at the table.

This will be the table on which the GROUP BY query will operate.

We order by `restName` to visually illustrate the groups that will be created.

In [15]:
%%read_sql
SELECT *
FROM Restaurant R INNER JOIN Rating T ON R.restCode=T.restCode
ORDER BY restName

Query started at 04:35:14 PM UTC; Query executed in 0.01 m

,restCode,restName,cuisine,borough,yearEst,avgPrice,code,cID,restCode,starRating,ratingDate,comments
0,111.0,Antonio Trattoria,Italian,Bronx,2008,75.0,None,NaN,NaN,NaN,None,None
1,103.0,Carbone,Italian,Manhattan,2010,150.0,R4,203.0,103.0,2.0,2015-02-01,Disappointed
2,103.0,Carbone,Italian,Manhattan,2010,150.0,R8,205.0,103.0,3.0,2012-02-16,None
3,105.0,Don Peppe,Italian,Queens,1998,75.0,None,NaN,NaN,NaN,None,None
4,108.0,Ela Taverna,Greek,Brooklyn,2015,150.0,R10,205.0,108.0,5.0,None,Must try fish
5,108.0,Ela Taverna,Greek,Brooklyn,2015,150.0,R5,203.0,108.0,4.0,2016-03-01,Great fish
6,108.0,Ela Taverna,Greek,Brooklyn,2015,150.0,R6,203.0,108.0,2.0,2018-06-30,Not as good as before
7,110.0,Han Dynasty,Chinese,Manhattan,2012,125.0,None,NaN,NaN,NaN,None,None
8,104.0,Il Mulino,Italian,Manhattan,1999,250.0,R13,208.0,104.0,3.0,2003-06-30,Overpriced
9,104.0,Il Mulino,Italian,Manhattan,1999,250.0,R14,209.0,104.0,3.0,1005-07-30,None


Now notice the output when we use a LEFT JOIN instead of an INNER JOIN. Notice the extra restaurants that appear, which have received no reviews (and therefore the `code` and `cID` are NULL)

In [16]:
%%read_sql
SELECT *
FROM Restaurant R LEFT JOIN Rating T ON R.restCode=T.restCode
ORDER BY restName

Query started at 04:36:27 PM UTC; Query executed in 0.01 m

,restCode,restName,cuisine,borough,yearEst,avgPrice,code,cID,restCode,starRating,ratingDate,comments
0,111.0,Antonio Trattoria,Italian,Bronx,2008,75.0,None,NaN,NaN,NaN,None,None
1,103.0,Carbone,Italian,Manhattan,2010,150.0,R4,203.0,103.0,2.0,2015-02-01,Disappointed
2,103.0,Carbone,Italian,Manhattan,2010,150.0,R8,205.0,103.0,3.0,2012-02-16,None
3,105.0,Don Peppe,Italian,Queens,1998,75.0,None,NaN,NaN,NaN,None,None
4,108.0,Ela Taverna,Greek,Brooklyn,2015,150.0,R10,205.0,108.0,5.0,None,Must try fish
5,108.0,Ela Taverna,Greek,Brooklyn,2015,150.0,R5,203.0,108.0,4.0,2016-03-01,Great fish
6,108.0,Ela Taverna,Greek,Brooklyn,2015,150.0,R6,203.0,108.0,2.0,2018-06-30,Not as good as before
7,110.0,Han Dynasty,Chinese,Manhattan,2012,125.0,None,NaN,NaN,NaN,None,None
8,104.0,Il Mulino,Italian,Manhattan,1999,250.0,R13,208.0,104.0,3.0,2003-06-30,Overpriced
9,104.0,Il Mulino,Italian,Manhattan,1999,250.0,R14,209.0,104.0,3.0,1005-07-30,None


We now execute the GROUP BY. Notice how we use the `COUNT` command to count the total number of reviews, and the reviews with comments. Since this is an `INNER JOIN`, the `COUNT(*)` and `COUNT(code)`

In [19]:
%%read_sql
SELECT restName, 
    COUNT(*) AS cnt, 
    COUNT(code) AS num_reviews, 
    COUNT(comments) AS num_reviews_with_comments
FROM Restaurant R INNER JOIN Rating T ON R.restCode=T.restCode
GROUP BY restName

Query started at 04:38:39 PM UTC; Query executed in 0.01 m

,restName,cnt,num_reviews,num_reviews_with_comments
0,Carbone,2,2,1
1,Ela Taverna,3,3,3
2,Il Mulino,3,3,1
3,Loukoumi Taverna,2,2,2
4,Nisi,1,1,1
5,Pok Pok,3,3,2


Now, let's switch to a LEFT JOIN. Notice an important change. We cannot rely on `COUNT(*)` anymore to count the number of reviews, and we need to be using the `COUNT(code)`. For example `Nisi` has one review: both the `cnt` and the `num_reviews` column are 1. However, for the `Don Peppe`, which has no reviews, we see that `cnt` is still 1, but `num_reviews` is 0.

In [20]:
%%read_sql
SELECT restName, 
    COUNT(*) AS cnt, 
    COUNT(code) AS num_reviews, 
    COUNT(comments) AS num_reviews_with_comments
FROM Restaurant R LEFT JOIN Rating T ON R.restCode=T.restCode
GROUP BY restName

Query started at 04:39:13 PM UTC; Query executed in 0.01 m

,restName,cnt,num_reviews,num_reviews_with_comments
0,Antonio Trattoria,1,0,0
1,Carbone,2,2,1
2,Don Peppe,1,0,0
3,Ela Taverna,3,3,3
4,Han Dynasty,1,0,0
5,Il Mulino,3,3,1
6,Jianbing Company,1,0,0
7,Kiin Thai,1,0,0
8,Loukoumi Taverna,2,2,2
9,Nisi,1,1,1


## Question 2: For every Manhattan restaurant output its name and the number of reviews submitted for this restaurant.

In [21]:
%%read_sql
SELECT restName, COUNT(code) AS num_reviews, COUNT(comments) AS num_reviews_with_comments
FROM Restaurant R LEFT JOIN Rating T ON R.restCode=T.restCode
WHERE borough = 'Manhattan'
GROUP BY restName

Query started at 04:42:54 PM UTC; Query executed in 0.01 m

,restName,num_reviews,num_reviews_with_comments
0,Carbone,2,1
1,Han Dynasty,0,0
2,Il Mulino,3,1
3,Kiin Thai,0,0
4,Nisi,1,1


## Question 3: For every restaurant that was reviewed more than once output it name and the number or reviews submitted for this restaurant.

In [22]:
%%read_sql
SELECT restName, COUNT(code) AS num_reviews, COUNT(comments) AS num_reviews_with_comments
FROM Restaurant R LEFT JOIN Rating T ON R.restCode=T.restCode
GROUP BY restName
HAVING num_reviews>1

Query started at 04:43:09 PM UTC; Query executed in 0.01 m

,restName,num_reviews,num_reviews_with_comments
0,Carbone,2,1
1,Ela Taverna,3,3
2,Il Mulino,3,1
3,Loukoumi Taverna,2,2
4,Pok Pok,3,2


## Question 4: Output the critic's last name and the restaurant name together with the maximal star rating ever issued by this critic for this restaurant.

Again, let's execute first the JOIN of all the tables that we need, so that we can see the data that we will be aggregating. Since we want to organize by critic's last name and restaurant name, we also add the corresponding ORDER BY, so that we can visually group together the rows that we will be aggregating.

In [27]:
%%read_sql
SELECT *
FROM Critic C 
  INNER JOIN Rating R ON R.cID = C.cID
  INNER JOIN Restaurant T ON T.restCode = R.restCode
ORDER BY restName, lastN

Query started at 04:50:44 PM UTC; Query executed in 0.01 m

,cID,firstN,lastN,affiliation,code,cID,restCode,starRating,ratingDate,comments,restCode,restName,cuisine,borough,yearEst,avgPrice
0,203.0,Brittany,Harris,Vogue,R4,203.0,103.0,2.0,2015-02-01,Disappointed,103.0,Carbone,Italian,Manhattan,2010,150.0
1,205.0,Chris,Jackson,NYT,R8,205.0,103.0,3.0,2012-02-16,None,103.0,Carbone,Italian,Manhattan,2010,150.0
2,203.0,Brittany,Harris,Vogue,R5,203.0,108.0,4.0,2016-03-01,Great fish,108.0,Ela Taverna,Greek,Brooklyn,2015,150.0
3,203.0,Brittany,Harris,Vogue,R6,203.0,108.0,2.0,2018-06-30,Not as good as before,108.0,Ela Taverna,Greek,Brooklyn,2015,150.0
4,205.0,Chris,Jackson,NYT,R10,205.0,108.0,5.0,None,Must try fish,108.0,Ela Taverna,Greek,Brooklyn,2015,150.0
5,209.0,George,Clarke,None,R14,209.0,104.0,3.0,1005-07-30,None,104.0,Il Mulino,Italian,Manhattan,1999,250.0
6,205.0,Chris,Jackson,NYT,R9,205.0,104.0,2.0,2000-02-16,None,104.0,Il Mulino,Italian,Manhattan,1999,250.0
7,208.0,Ashley,White,NYT,R13,208.0,104.0,3.0,2003-06-30,Overpriced,104.0,Il Mulino,Italian,Manhattan,1999,250.0
8,202.0,Daniel,Lewis,WP,R3,202.0,106.0,4.0,None,"Great atmosphere, friendly staff",106.0,Loukoumi Taverna,Greek,Queens,1994,130.0
9,206.0,Elizabeth,Thomas,Chronicle,R12,206.0,106.0,5.0,2001-12-21,Loved everything,106.0,Loukoumi Taverna,Greek,Queens,1994,130.0


Now, let's report the number of reviews that a critic wrote for the restaurant, together with the mix and max ratings. Obviously, when there is only one review, the min and max ratings are the same.

In [28]:
%%read_sql
SELECT restName, lastN, 
    COUNT(R.code) AS num_reviews, 
    MAX(R.starRating) AS maxRating, 
    MIN(R.starRating) AS minRating 
FROM Critic C 
  INNER JOIN Rating R ON R.cID = C.cID
  INNER JOIN Restaurant T ON T.restCode = R.restCode
GROUP BY restName, lastN
ORDER BY restName, lastN

Query started at 04:53:30 PM UTC; Query executed in 0.01 m

,restName,lastN,num_reviews,maxRating,minRating
0,Carbone,Harris,1,2.0,2.0
1,Carbone,Jackson,1,3.0,3.0
2,Ela Taverna,Harris,2,4.0,2.0
3,Ela Taverna,Jackson,1,5.0,5.0
4,Il Mulino,Clarke,1,3.0,3.0
5,Il Mulino,Jackson,1,2.0,2.0
6,Il Mulino,White,1,3.0,3.0
7,Loukoumi Taverna,Lewis,1,4.0,4.0
8,Loukoumi Taverna,Thomas,1,5.0,5.0
9,Nisi,Thomas,1,3.0,3.0


## Question 5: For each cuisine-borough pair, output the number of the corresponding restaurants.

In [0]:
%%read_sql


## Question 6: For each NYT reporter, output the number of distinct restaurants this reporter reviewed.

In [0]:
%%read_sql


## Question 7: For every news outlet, output the average star rating submitted by all the reviewers of this outlet. a. consider only Italian restaurants b. consider only Italian restaurants outside of Manhattan

In [0]:
%%read_sql


## Question 8: For every borough output the max star rating submitted for any restaurant within this borough (in which borough do you have the best restaurant)

In [0]:
%%read_sql
